In [50]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)
from functions import directionOfEffect
from pyspark.ml.functions import array_to_vector, vector_to_array

spark = SparkSession.builder.getOrCreate()

#### Can we use mouse phenotypes for: 
    >> Predict Safety Liabilities
    >> Drug warnings (Black Box Warnings)
    >> Withdrawn drugs

In [23]:
mophe_path = "gs://open-targets-data-releases/24.09/output/etl/parquet/mousePhenotypes"
mophe = spark.read.parquet(mophe_path)

target_path = "gs://open-targets-data-releases/24.09/output/etl/parquet/targets/"
target = spark.read.parquet(target_path)

mopheScore_path = "gs://ot-team/jroldan/20230825_mousePheScores.csv"
mopheScore = spark.read.csv(mopheScore_path, header=True)

#### Make dataset of targets with safety Liabilities.
    There are multiple safety liabitily types

>> See how many differents datasource of safety liabilities 

In [12]:
(
    target.select("id", F.explode_outer(F.col("safetyLiabilities")).alias("safeLiable"))
    .select("id", "safeLiable.*")
    .groupBy("datasource")
    .count()
).show(truncate=False)

+---------------------+-----+
|datasource           |count|
+---------------------+-----+
|Force et al. (2011)  |47   |
|null                 |62180|
|Lamore et al. (2017) |30   |
|Brennan et al. (2024)|210  |
|AOP-Wiki             |227  |
|Lynch et al. (2017)  |1341 |
|Bowes et al. (2012)  |313  |
|Urban et al. (2012)  |254  |
|ToxCast              |375  |
|PharmGKB             |1661 |
+---------------------+-----+



#### Remove datasources of Brennan et al (2024) and PharmGKB (Pharmacogenetics)


#### safety liabilities without Brennan and PGx

In [60]:
safetyLiability = (
    target.select("id", F.explode_outer(F.col("safetyLiabilities")).alias("safeLiable"))
    .select("id", "safeLiable.*")
    .filter(~F.col("datasource").isin(["Brennan et al. (2024)", "PharmGKB"]))
    .select("id", "event")
    .groupBy("id")
    .agg(F.count("event").alias("safetyLiabl"))
    .sort(F.col("safetyLiabl").asc())
)
n = safetyLiability.count()
print("There are", n, "targets with safety liabilities excluding Brennan 2024 and PGx")

There are 503 targets with safety liabilities excluding Brennan 2024 and PGx


#### genetic constraint and lof_tolerance score

In [51]:
constr = (
    target.select("id", F.explode_outer(F.col("constraint")).alias("genConstraint"))
    .select("id", "genConstraint.*")
    .filter(
        F.col("constraintType")
        == "lof"
        # ).sort(F.col("upperRank").desc()
    )
    .select(
        "id",
        "upperRank",
        "oe",
        F.col("score").alias("scoreConstraint"),
    )
)
minUpperRank = (
    target.select(F.col("id").alias("constr_id"), F.explode("constraint"))
    .select(F.col("col.*"))
    .filter(F.col("constraintType") == "lof")
    .groupBy("constraintType")
    .agg(F.min("upperRank").alias("upperRank"))
    .select("upperRank")
    .rdd.flatMap(lambda x: x)
    .collect()[0]
)

maxUpperRank = (
    target.select(F.col("id").alias("constr_id"), F.explode("constraint"))
    .select(F.col("col.*"))
    .filter(F.col("constraintType") == "lof")
    .groupBy("constraintType")
    .agg(F.max("upperRank").alias("upperRank"))
    .select("upperRank")
    .rdd.flatMap(lambda x: x)
    .collect()[0]
)

loftolerance = (
    target.select(F.col("id").alias("constr_id"), F.explode("constraint"))
    .select(F.col("constr_id"), F.col("col.*"))
    .filter(F.col("constraintType") == "lof")
    .withColumn(
        "cal_score",
        F.lit(
            (2 * ((F.col("upperRank") - minUpperRank) / (maxUpperRank - minUpperRank)))
            - 1
        ),
    )
    .selectExpr("constr_id as id", "cal_score", "constraintType")
)

#### mouse score

In [29]:
mousePhenoScoreFilter = mopheScore.select(
    F.col("id").alias("idLabel"),
    F.col("label").alias("phenoLabel"),
    F.col("score"),
).withColumn(
    "curatedScore",
    F.when(F.col("score") == 0.0, F.lit(0)).otherwise(F.lit(F.col("score"))),
)

#### mouse score per target

In [30]:
scoreCalc_list = (
    mophe.select(
        "targetFromSourceId",
        F.explode_outer(F.col("modelPhenotypeClasses.id")).alias("id"),
    )
    .join(mousePhenoScoreFilter, F.col("id") == mousePhenoScoreFilter.idLabel, "left")
    ##.na.drop(subset=['scoreRevisado3'])
    .withColumn("score", F.col("curatedScore").cast(FloatType()))
    .groupBy("targetFromSourceId")
    .agg(array_to_vector(F.collect_list("score")).alias("score"))
    #    .join(constr.select("constr_id","upperRank"), constr.constr_id == F.col("targetFromSourceId"),"left")
)

#### Safety WO ToxCast

In [32]:
safetyWOToxCast = (
    target.select("id", F.explode_outer(F.col("safetyLiabilities")).alias("safeLiable"))
    .filter(
        ~F.col("safeLiable.datasource").isin(
            ["ToxCast", "Brennan et al. (2024)", "PharmGKB"]
        )
    )
    .select("id", "safeLiable.*")
    .select("id", "event")
    .groupBy("id")
    .agg(F.count("event").alias("noToxCast"))
    .sort(F.col("noToxCast").asc())
)

n = safetyWOToxCast.count()
print(
    "There are",
    n,
    " targets with safety liabilities excluding ToxCast, Brennan 2024 and PGx",
)

There are 262  targets with safety liabilities excluding ToxCast, Brennan 2024 and PGx


#### approved Drugs

In [33]:
approvedTargets = spark.read.csv(
    "gs://ot-team/jroldan/2013-2022_approvals.csv", header=True
).drop("_c0")
approved = approvedTargets.select("targetIds")


def remove_all_whitespace(col):
    return F.regexp_replace(col, "\\s+", "")


approvedTargets = (
    approved.withColumn("targets", (F.explode(F.split(F.col("targetIds"), ";"))))
    .withColumn("trimmed", F.trim(F.col("targets")))
    .selectExpr("trimmed as id")
    .withColumn("approved", F.lit("approved"))
    .distinct()
)

### Calculate Score from mouse phenotypes using harmonic sum

In [34]:
df_py = scoreCalc_list.toPandas()

values = []
for row in df_py["score"]:
    z = sorted(row, reverse=True)
    values.append(harmonic_sum(z))

maximumScore = 1.644

df_py["harmonicSum"] = values
normalised = []
for row in df_py["harmonicSum"]:
    new = row / (maximumScore)
    normalised.append(new)
df_py["harmonicSumNorm"] = normalised

## convert pandas to spark dataframe
df = spark.createDataFrame(df_py).withColumnRenamed("targetFromSourceId", "id")

#### Run DoE to have LoF and GoF drugs

In [7]:
evidences = (
    spark.read.parquet(
        "gs://open-targets-data-releases/24.06/output/etl/parquet/evidence"
    )
    .filter(
        F.col("datasourceId").isin(
            [
                "ot_genetics_portal",
                "gene_burden",
                "eva",
                "eva_somatic",
                "gene2phenotype",
                "orphanet",
                "cancer_gene_census",
                "intogen",
                "impc",
                "chembl",
            ]
        )
    )
    .persist()
)
platform_v = "24.09"
dataset = directionOfEffect(evidences, platform_v).persist()

24/09/27 19:57:58 WARN CacheManager: Asked to cache already cached data.
24/09/27 19:57:59 WARN CacheManager: Asked to cache already cached data.
24/09/27 19:57:59 WARN CacheManager: Asked to cache already cached data.


#### Load Drug warnings to take Black Box Warning Drugs and Withdrawn Drugs

In [35]:
drugWarning_path = (
    "gs://open-targets-data-releases/24.09/output/etl/parquet/drugWarnings"
)
drugwarnings = spark.read.parquet(drugWarning_path)

#### filter Drugs by inhibitors (LoF)
lofDrugs = (
    dataset.filter(F.col("datasourceId") == "chembl")
    .select("targetId", "drugId", "homogenized")
    .filter(F.col("homogenized") == "LoF_protect")
    .groupBy("targetId", "drugId")
    .count()
)

lofDrugsWarnings = lofDrugs.join(
    drugwarnings.withColumn("drugId", F.explode_outer("chemblIds")).select(
        "drugId", "toxicityClass", "warningType"
    ),
    on="drugId",
    how="left",
).distinct()

#### From Drugs with BBW to Targets with BBW

In [36]:
suffix_bbw = "_BBW"

lofDrugWarningBBX = (
    lofDrugsWarnings.filter(F.col("warningType") == "Black Box Warning")
    .groupBy("targetId")
    .pivot("toxicityClass")
    .agg(F.collect_set("warningType"))
)

array_columns = lofDrugWarningBBX.columns[1:]

# The value to check in the arrays
value_to_check = "Black Box Warning"

# Create a new DataFrame with transformed columns
df_transformed = lofDrugWarningBBX

for col_name in array_columns:
    new_col_name = f"{col_name}_BBW"
    df_transformed = df_transformed.withColumn(
        new_col_name,
        F.when(F.array_contains(F.col(col_name), value_to_check), 1).otherwise(0),
    )

lofDrugWarningBBX_format = df_transformed.drop(*array_columns).withColumn(
    "allBBW", F.lit(1)
)

#### From Drugs with WithDrawn to Targets with withDrawn

In [37]:
lofDrugWarningWD = (
    lofDrugsWarnings.filter(F.col("warningType") == "Withdrawn")
    .groupBy("targetId")
    .pivot("toxicityClass")
    .agg(F.collect_set("warningType"))
)

array_columns = lofDrugWarningWD.columns[1:]

# The value to check in the arrays
value_to_check = "Withdrawn"

# Create a new DataFrame with transformed columns
df_transformed = lofDrugWarningWD

for col_name in array_columns:
    new_col_name = f"{col_name}_WD"
    df_transformed = df_transformed.withColumn(
        new_col_name,
        F.when(F.array_contains(F.col(col_name), value_to_check), 1).otherwise(0),
    )

lofDrugWarningWD_format = df_transformed.drop(*array_columns).withColumn(
    "allWD", F.lit(1)
)

#### Join Drugs BBW and Drugs WithDrawn

In [38]:
lofDrugsBBw_WD = (
    lofDrugWarningWD_format.join(lofDrugWarningBBX_format, on="targetId", how="outer")
    .withColumn("bwwAndwd", F.lit(1))
    .na.fill(0)
)

### Make dataset and prepare normalized harmonic sum 

In [39]:
### make dataset for comparisons
df_comparisons = (
    df.join(loftolerance, on="id", how="left")
    .join(safetyLiability, on="id", how="left")
    .join(safetyWOToxCast, on="id", how="left")
    .join(approvedTargets, on="id", how="left")
    .join(lofDrugsBBw_WD.withColumnRenamed("targetId", "id"), on="id", how="left")
    .withColumn("allBBW", F.when(F.col("allBBW") == 1, F.lit(1)).otherwise(F.lit(0)))
    .withColumn("allWD", F.when(F.col("allWD") == 1, F.lit(1)).otherwise(F.lit(0)))
    .persist()
)

#### Make deciles for harmonic sum
# Calculate quartiles or deciles
quantiles = df_comparisons.approxQuantile(
    "harmonicSumNorm", [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 0.01
)
# Define quartile or decile bins
bins = [float("-inf")] + quantiles + [float("inf")]
window_spec = Window.orderBy("harmonicSumNorm")
df_with_labels = (
    df_comparisons.withColumn(
        "decilesHarmonicSumNorm",
        F.when(F.col("harmonicSumNorm").isNull(), None).otherwise(
            sum(
                F.when(F.col("harmonicSumNorm") >= bin_val, 1).otherwise(0)
                for bin_val in bins
            )
        ),
    )
    .withColumn(
        "safetyLiabilities", F.when(F.col("nr").isNotNull(), F.lit(1)).otherwise(0)
    )
    .withColumn(
        "noToxCastLiab", F.when(F.col("noToxCast").isNotNull(), F.lit(1)).otherwise(0)
    )
    .drop("nr", "noToxCast")
)

#### Use human genetics to define the Severity Score

> Define disease types (Rare or Common)

In [41]:
## rare diseases
rare = [
    "eva",
    "cancer_gene_census",
    "eva_somatic",
    "orphanet",
    "genomics_england",
    "gene2phenotype",
    "intogen",
]

## common diseases
common = ["ot_genetics_portal", "gene_burden"]

In [43]:
diseaseTypes = (  ### association of target with diseases caused by (LoF risk)
    dataset.withColumn(
        "diseaseType",
        F.when(F.col("datasourceId").isin(rare), F.lit("rare")).when(
            F.col("datasourceId").isin(common), F.lit("common")
        ),
    )
    .filter(
        (
            F.col("homogenized") == "LoF_risk"
        )  ### filter by diseases caused by targets with LoF mutations
        & (F.col("diseaseType").isNotNull())
    )
    .groupBy("targetId", "diseaseId")
    .agg(
        F.count("targetId").alias("nr"),
        F.collect_set("diseaseType").alias("diseaseType"),
    )
    .withColumn("bivalent", F.size(F.col("diseaseType")))
    # .filter(F.col("bivalent") == 1)
).persist()

#### approach rare = severe, common = non severe, rare&common = mid severe

In [44]:
scoring = (
    diseaseTypes.groupBy("targetId")
    .agg(F.flatten(F.collect_set("diseaseType")).alias("diseaseType"))
    .withColumn(
        "scoreFromDisease",
        F.when(
            (F.size(F.col("diseaseType")) == 1)
            & (F.array_contains(F.col("diseaseType"), "common")),
            F.lit(0.25),
        )
        .when(
            (F.size(F.col("diseaseType")) == 1)
            & (F.array_contains(F.col("diseaseType"), "rare")),
            F.lit(1),
        )
        .when(F.size(F.col("diseaseType")) == 2, F.lit(0.7))
        .when(F.size(F.col("diseaseType")) == 0, F.lit(0)),
    )
)

In [55]:
phenoDiseases = (
    mophe.select("targetFromSourceId", "modelPhenotypeLabel", "targetFromSourceId")
    .withColumnRenamed("targetFromSourceId", "targetId")
    .join(scoring, on="targetId", how="right")
    .withColumn("typesOfDiseases", F.explode_outer(F.col("diseaseType")))
    .groupBy("modelPhenotypeLabel")
    .agg(F.collect_set("typesOfDiseases").alias("diseaseType"))
    .withColumn(
        "scoreFromDisease",
        F.when(
            (F.size(F.col("diseaseType")) == 1)
            & (F.array_contains(F.col("diseaseType"), "common")),
            F.lit(0.2),
        )
        .when(
            (F.size(F.col("diseaseType")) == 1)
            & (F.array_contains(F.col("diseaseType"), "rare")),
            F.lit(1),
        )
        .when(
            (F.size(F.col("diseaseType")) == 2)
            & (F.col("modelPhenotypeLabel").isNotNull()),
            F.lit(0.6),
        )
        .when(F.size(F.col("diseaseType")) == 0, F.lit(0))
        .otherwise(F.lit(0)),
    )
)

scoreDiseases1 = (
    mophe.select("targetFromSourceId", "modelPhenotypeLabel")
    .join(phenoDiseases, on="modelPhenotypeLabel", how="right")
    .groupBy("targetFromSourceId")
    .agg(array_to_vector(F.collect_list("scoreFromDisease")).alias("score"))
)

df_py = scoreDiseases1.toPandas()

values = []
for row in df_py["score"]:
    z = sorted(row, reverse=True)
    values.append(harmonic_sum(z))
# maximumScore = max(values)

maximumScore = 1.644

df_py["harmonicSum"] = values
normalised = []
for row in df_py["harmonicSum"]:
    new = row / (maximumScore)
    normalised.append(new)
df_py["harmonicSumNorm"] = normalised


## convert pandas to spark dataframe
df_scoreDisease1 = (
    spark.createDataFrame(df_py)
    .withColumnRenamed("targetFromSourceId", "id")
    .selectExpr(
        "id",
        "score as scoreDiseases1",
        "harmonicSumNorm as dis1HarmSumNorm",
    )
    .filter(F.col("id").isNotNull())
)

df_comparisons2 = df_comparisons.join(df_scoreDisease1, on="id", how="left")

> Score regarding the evidences relating the target with the disease

In [57]:
phenoDiseases2 = (
    evidences.filter(F.col("datasourceId") == "impc")
    .withColumn(
        "explotedPhenotypes",
        F.explode_outer(F.col("diseaseModelAssociatedModelPhenotypes")),
    )
    .select("targetId", "diseaseId", "explotedPhenotypes")
    .join(
        diseaseTypes.select("diseaseId", "diseaseType", "bivalent"),
        on="diseaseId",
        how="left",
    )
    .withColumn(
        "scoreFromDisease",
        F.when(
            (F.size(F.col("diseaseType")) == 1)
            & (F.array_contains(F.col("diseaseType"), "common")),
            F.lit(0.2),
        )
        .when(
            (F.size(F.col("diseaseType")) == 1)
            & (F.array_contains(F.col("diseaseType"), "rare")),
            F.lit(1),
        )
        .when(
            (F.size(F.col("diseaseType")) == 2),
            F.lit(0.6),
        )
        .when(F.size(F.col("diseaseType")) == 0, F.lit(0))
        .otherwise(F.lit(0)),
    )
    .groupBy("explotedPhenotypes")
    .agg(F.collect_set("scoreFromDisease").alias("scores"))
    .withColumn("NrScores", F.size("scores"))
    .withColumn(
        "average",
        F.expr("aggregate(scores, 0D, (acc, x) -> acc + x, acc -> acc / size(scores))"),
    )
).persist()

scoreDiseases2 = (
    mophe.select("targetFromSourceId", "modelPhenotypeLabel")
    .join(
        phenoDiseases2.selectExpr(
            "explotedPhenotypes.label as modelPhenotypeLabel", "average"
        ),
        on="modelPhenotypeLabel",
        how="right",
    )
    .groupBy("targetFromSourceId")
    .agg(array_to_vector(F.collect_list("average")).alias("score"))
)

df_py = scoreDiseases2.toPandas()

values = []
for row in df_py["score"]:
    z = sorted(row, reverse=True)
    values.append(harmonic_sum(z))
# maximumScore = max(values)

maximumScore = 1.644

df_py["harmonicSum"] = values
normalised = []
for row in df_py["harmonicSum"]:
    new = row / (maximumScore)
    normalised.append(new)
df_py["harmonicSumNorm"] = normalised


## convert pandas to spark dataframe
df_scoreDisease2 = (
    spark.createDataFrame(df_py)
    .withColumnRenamed("targetFromSourceId", "id")
    .selectExpr(
        "id",
        "score as scoreDiseases2",
        "harmonicSumNorm as dis2HarmSumNorm",
    )
    .filter(F.col("id").isNotNull())
)

df_comparisons3 = df_comparisons2.join(df_scoreDisease2, on="id", how="left")

## ROC Curves

In [44]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
import random

random.seed(42)
np.random.seed(42)

### ROC curve for predicting safety liabilities with score Diseases1

In [58]:
df_comparisons3.printSchema()

root
 |-- id: string (nullable = true)
 |-- score: vector (nullable = true)
 |-- harmonicSum: double (nullable = true)
 |-- harmonicSumNorm: double (nullable = true)
 |-- cal_score: double (nullable = true)
 |-- constraintType: string (nullable = true)
 |-- nr: long (nullable = true)
 |-- noToxCast: long (nullable = true)
 |-- approved: string (nullable = true)
 |-- null_WD: integer (nullable = true)
 |-- carcinogenicity_WD: integer (nullable = true)
 |-- cardiotoxicity_WD: integer (nullable = true)
 |-- dermatological toxicity_WD: integer (nullable = true)
 |-- gastrointestinal toxicity_WD: integer (nullable = true)
 |-- hematological toxicity_WD: integer (nullable = true)
 |-- hepatotoxicity_WD: integer (nullable = true)
 |-- immune system toxicity_WD: integer (nullable = true)
 |-- metabolic toxicity_WD: integer (nullable = true)
 |-- misuse_WD: integer (nullable = true)
 |-- musculoskeletal toxicity_WD: integer (nullable = true)
 |-- nephrotoxicity_WD: integer (nullable = true)
 |-

In [ ]:
### roc curve for Safety Liabilities containing ToxCast:
targetPredictedToxCastNO = df_comparisons3.filter(
    F.col("safetyLiablEventN") == 1
).withColumn("y", F.lit(1))
targetWOPredictedToxCastNo = (
    df_with_labels8.filter(F.col("safetyLiablEventN") == 0)
    .sample(False, 0.5, seed=0)
    .limit(469)
    .withColumn("y", F.lit(0))
)

In [ ]:
### harmonicsumnormDiseases1

targetsToCheckTargetNoToxCast = targetPredictedToxCastNO.union(
    targetWOPredictedToxCastNo
)
## convert to array WO ToxCast
y_score_targetPredictToxCast = np.array(
    targetsToCheckTargetNoToxCast.select("harmonicSumDiseases1")
    .rdd.flatMap(lambda x: x)
    .collect()
)
y_targetPredicttoxCast = np.array(
    targetsToCheckTargetNoToxCast.select("y").rdd.flatMap(lambda x: x).collect()
)

### ROc curve
## predicted targets
fpr_noToxCastTarget, tpr_noToxCastTarget, thresholds = roc_curve(
    y_targetPredicttoxCast, y_score_targetPredictToxCast
)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(fpr_noToxCastTarget, tpr_noToxCastTarget)
ax.plot([0, 0.5, 1], [0, 0.5, 1])
ax.fill_between(fpr_noToxCastTarget, tpr_noToxCastTarget, step="pre", alpha=0.4)
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")
ax.set_title("ROC curve for targets with safety Liabilities (including ToxCast)")
ax.legend(
    [
        "HarmonicSumNorm as Predictor",
        "Random Classifier",
        "HarmonicSum - AUC {:.3f}".format(
            roc_auc_score(y_targetPredicttoxCast, y_score_targetPredictToxCast)
        ),
    ]
)
plt.show()